In [1]:
# Importing libraries
import pandas as pd
from datetime import datetime
import time
import random

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
# Defining parameters for controllable pitch
PICK_PITCH = 'pitch_1'

In [ ]:
# Defining parameters for login details
SMTP_SERVER = 'smtp.zoho.in'
SMTP_PORT = 587
EMAIL = 'hello@antryl.com'
APP_PASSWORD = '********'

In [4]:
# Defining HTML email body
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Antryl Consulting</title>
</head>

<body style="margin:0; padding:0; background-color:#f4f6f8;">
  <table width="100%" cellpadding="0" cellspacing="0" bgcolor="#f4f6f8">
    <tr>
      <td align="center" style="padding:40px 0;">

        <!-- Main Card -->
        <table width="600" cellpadding="0" cellspacing="0" bgcolor="#ffffff"
        style="
            border-radius:10px;
            box-shadow:0 6px 16px rgba(0,0,0,0.08);
            border-top:4px solid #5468FF;
        ">

        <!-- Body -->
        <tr>
        <td style="
            padding:36px 40px 32px 40px;
            font-family:Arial, Helvetica, sans-serif;
            font-size:14px;
            line-height:1.7;
            color:#1f2933;
        ">
            {{body}}
        </td>
        </tr>

        <!-- Divider -->
        <tr>
        <td style="padding:0 40px;">
            <hr style="
            border:none;
            border-top:1px solid #e5e7eb;
            margin:0;
            ">
        </td>
        </tr>

        <!-- Signature -->
        <tr>
        <td style="
            padding:24px 40px 36px 40px;
            font-family:Arial, Helvetica, sans-serif;
            font-size:13px;
            line-height:1.6;
            color:#374151;
        ">

        <!-- Logo -->
        <img src="https://raw.githubusercontent.com/antryl/antryl.github.io/refs/heads/main/assets/img/logo/logo.png"
            alt="Antryl Consulting"
            width="100"
            style="display:block; margin-bottom:14px;
        ">

        <strong style="color:#111827;">Antryl Consulting</strong><br>
        Your On-Demand Analytics Partner<br>

        <a href="https://www.antryl.com"
            style="
            color:#5468FF;
            text-decoration:none;
            font-weight:500;
            ">
            www.antryl.com
        </a>
        </td>
        </tr>

        </table>
        <!-- End Main Card -->

      </td>
    </tr>
  </table>
</body>
</html>
"""

In [5]:
# Function to send email
def send_email(to_email, subject, html_body):
    msg = MIMEMultipart('alternative')
    msg['From'] = EMAIL
    msg['To'] = to_email
    msg['Subject'] = subject

    # Plain-text fallback
    text_fallback = 'This email requires HTML support.'

    msg.attach(MIMEText(text_fallback, 'plain'))
    msg.attach(MIMEText(html_body, 'html'))

    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(EMAIL, APP_PASSWORD)
        server.sendmail(EMAIL, to_email, msg.as_string())

In [9]:
# Get lead's information
leads = (
    pd.read_excel('C:\\Users\\manuj\\Downloads\\bulk-application-sender\\data\\leads.xlsx', sheet_name='pitch_1')
    .query('flag == 1')
)

In [11]:
# Read previous log
logs = (
    pd.read_excel('C:\\Users\\manuj\\Downloads\\bulk-application-sender\\data\\logs.xlsx', sheet_name='logs')
)

In [12]:
# Get email pitch and subject
email_pitch = (
    pd.read_excel('C:\\Users\\manuj\\Downloads\\bulk-application-sender\\templates\\email_pitch.xlsx', sheet_name='pitch')
)

pitch_body = (
    email_pitch
    .query('pitch_id == @PICK_PITCH')['pitch_description']
    .reset_index(drop=True)[0] 
)

pitch_subject = (
    email_pitch
    .query('pitch_id == @PICK_PITCH')['subject']
    .reset_index(drop=True)[0]
)

In [13]:
# Function to get personalized pitch subject and body
def get_personalized_info(pitch, row):
    for key, value in row.items():
        pitch = pitch.replace(f"{{{{{key}}}}}", str(value))
    return pitch

In [14]:
# Functions to build html body for email
def text_to_html_paragraphs(text):
    paragraphs = text.strip().split("\n\n")
    return "".join(
        f"<p style='margin:0 0 16px 0;'>{p}</p>"
        for p in paragraphs
    )

def build_email_html(pitch_text):
    return HTML_TEMPLATE.replace(
        "{{body}}",
        text_to_html_paragraphs(pitch_text)
    )

In [15]:
# Required variable
log_buffer = []

# Function to prepare logs
def add_logs(email, current_timestamp, pitch_used):
    log_buffer.append({
        'email': email,
        'sent_at': current_timestamp,
        'last_pitch_used': pitch_used
    })

In [ ]:
# Trigger pitch
for _, row in leads.iterrows():
    try:
        subject = get_personalized_info(pitch_subject, row)
        pitch_text = get_personalized_info(pitch_body, row)
        html_body = build_email_html(pitch_text)

        # Trigger email
        send_email(
            row["email"],
            subject,
            html_body
        )

        # Delay time in between two emails
        time.sleep(random.randint(45, 60))

        # Buffer log
        add_logs(
            row['email'],
            pd.Timestamp('now').strftime('%Y-%m-%d %H:%M:%S'),
            PICK_PITCH
        )

    except Exception as e:
        print(f"Failed for {row['email']}: {e}")

# Append into logs sheet
log_dictionary = (
    pd.concat(
        [pd.DataFrame(log_buffer), logs],
        ignore_index=True
    )
)
log_dictionary.to_excel('C:\\Users\\manuj\\Downloads\\bulk-application-sender\\data\\logs.xlsx', sheet_name='logs', index=False)